In [56]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import pandas as pd
import numpy as np

In [57]:
data = pd.read_csv(r"C:\Users\Mayur\Documents\College\4th sem\Exploratory\Data\Sentinel1_MODIS_SM_Masked_Urban_YellowRiver_11km.csv")

In [58]:
def clean_data(data):
    # Drop rows with missing data in columns: 'LAI', 'SoilMoisture' and 3 other columns
    data = data.dropna(subset=['LAI', 'SoilMoisture', 'VH', 'VV', 'date'])
    # Group by 'date' and calculate the average of numeric columns
    data = data.groupby('date').mean(numeric_only=True).reset_index()
    # Drop column: 'SoilRoughness_placeholder'
    data = data.drop(columns=['SoilRoughness_placeholder'])
    # Drop column: 'Frequency_GHz'
    data = data.drop(columns=['Frequency_GHz'])
    # Extract year, month, and day from the 'date' column
    data['Year'] = pd.to_datetime(data['date']).dt.year
    data['Month'] = pd.to_datetime(data['date']).dt.month
    data['Day'] = pd.to_datetime(data['date']).dt.day
    # Drop column: 'date'
    data = data.drop(columns=['date'])
    # Drop column: 'Year'
    data = data.drop(columns=['Year'])
    # Convert Month and Day columns to numeric
    data['Month'] = pd.to_numeric(data['Month'], errors='coerce')
    data['Day'] = pd.to_numeric(data['Day'], errors='coerce')
    # Add two new columns for Sin and Cos transformations of Month
    data['Month_Sin'] = np.sin(2 * np.pi * (data['Month'] / 12))
    data['Month_Cos'] = np.cos(2 * np.pi * (data['Month'] / 12))
    # Add sin and cos transformations of Day
    data['Day_Sin'] = np.sin(2 * np.pi * (data['Day'] / 30))
    data['Day_Cos'] = np.cos(2 * np.pi * (data['Day'] / 30))
    # Drop column: 'Day'
    data = data.drop(columns=['Day'])
    # Drop column: 'Month'
    data = data.drop(columns=['Month'])
    # Convert VV and VH from decibels to linear
    data['VV'] = 10 ** (data['VV'] / 10)
    data['VH'] = 10 ** (data['VH'] / 10)
    # Scale VV and VH normally
    scaler_vv_vh = StandardScaler()
    data[['VV', 'VH']] = scaler_vv_vh.fit_transform(data[['VV', 'VH']])
    # Scale SoilMoisture normally
    scaler_sm = StandardScaler()
    data['SoilMoisture'] = scaler_sm.fit_transform(data[['SoilMoisture']])
    # Scale IncidenceAngle with 1/10 importance of SoilMoisture
    data['IncidenceAngle'] = data['IncidenceAngle'] * 0.1 / data['SoilMoisture'].std()
    # Scale LAI with 0.75 importance of SoilMoisture
    data['LAI'] = data['LAI'] * 0.75 / data['SoilMoisture'].std()
    # Sin and Cos columns remain unchanged
    # (No operation needed for Month_Sin, Month_Cos, Day_Sin, Day_Cos)
    # Ensure the result is a DataFrame
    data = pd.DataFrame(data)
    return data


data_clean = clean_data(data.copy())
data_clean.head()

,IncidenceAngle,LAI,SoilMoisture,VH,VV,Month_Sin,Month_Cos,Day_Sin,Day_Cos
0,4.172704,0.372937,-0.926507,-0.316372,0.775638,1.224647e-16,-1.000000,0.866025,0.500000
1,3.552232,0.500332,-1.288683,0.919617,0.998619,1.224647e-16,-1.000000,-0.406737,-0.913545
2,3.553397,0.621952,-0.326103,4.863161,3.679401,1.224647e-16,-1.000000,-0.207912,0.978148
3,3.552746,1.093448,-0.563124,0.889493,1.093552,-5.000000e-01,-0.866025,0.743145,-0.669131
4,4.595357,1.166352,-0.805235,0.651093,0.753600,-5.000000e-01,-0.866025,-0.207912,-0.978148


In [59]:
feature_cols = ['VV', 'VH', 'IncidenceAngle', 'Month_Sin', 'Month_Cos', 'Day_Sin', 'Day_Cos']
target_cols = ['LAI', 'SoilMoisture']

X = data_clean[feature_cols].values
y = data_clean[target_cols].values

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [61]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

In [ ]:
# ----- WCM-Inspired Module -----
class WCMInspiredModule(nn.Module):
    def __init__(self):
        super(WCMInspiredModule, self).__init__()
        self.soil_transform = nn.Sequential(
            nn.Linear(1, 8), nn.ReLU(), nn.Linear(8, 1)
        )
        self.veg_transform = nn.Sequential(
            nn.Linear(1, 8), nn.ReLU(), nn.Linear(8, 1)
        )
        self.angle_transform = nn.Sequential(
            nn.Linear(1, 8), nn.ReLU(), nn.Linear(8, 1)
        )

    def forward(self, SM, LAI, IncAngle):
        soil_sig = self.soil_transform(SM)
        veg_attn = torch.exp(-F.relu(self.veg_transform(LAI)))
        angle_mod = self.angle_transform(IncAngle)
        sigma0 = soil_sig * veg_attn + angle_mod
        return sigma0

# ----- Full Model -----


class FullHybridModel(nn.Module):
    def __init__(self):
        super(FullHybridModel, self).__init__()
        self.phys_layer = WCMInspiredModule()
        self.mlp = nn.Sequential(
            nn.Linear(8, 64), nn.ReLU(),
            nn.Linear(64, 32), nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        IncAngle = x[:, 0:1]
        VH = x[:, 1:2]
        VV = x[:, 2:3]  # not used in forward, but could be added in inverse
        temporal = x[:, 3:]  # sin/cos of month/day

        # Dummy SM and LAI inputs, learnable placeholders
        dummy_SM = VV
        dummy_LAI = VH

        wcm_out = self.phys_layer(dummy_SM, dummy_LAI, IncAngle)
        x_cat = torch.cat([wcm_out, x], dim=1)
        return self.mlp(x_cat)


# ----- Training -----
model = FullHybridModel()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

n_epochs = 100
for epoch in range(n_epochs):
    model.train()
    optimizer.zero_grad()
    output = model(X_train_tensor)
    loss = criterion(output, y_train_tensor)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        model.eval()
        with torch.no_grad():
            val_output = model(X_test_tensor)
            val_r2 = r2_score(y_test_tensor.numpy(), val_output.numpy())
        print(f"Epoch {epoch}: Train Loss = {loss.item():.4f}, Val R^2 = {val_r2:.4f}")

# ----- Final R^2 -----
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor).numpy()
    r2 = r2_score(y_test_tensor.numpy(), y_pred)
    print(f"Final R^2: {r2:.4f}")

TypeError: linear(): argument 'input' (position 1) must be Tensor, not numpy.ndarray